In [ ]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 30.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers torchaudio pydub

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment

# Initialize the recognizer
recognizer = sr.Recognizer()

# Load the MP3 audio file and convert it to WAV format
mp3_audio_file = "/content/153955.mp3"
wav_audio_file = "converted_audio.wav"

audio = AudioSegment.from_mp3(mp3_audio_file)
audio = audio.set_frame_rate(16000)  # Adjust the sample rate as needed
audio.export(wav_audio_file, format="wav")

# Create a function to extract lyrics
recognizer = sr.Recognizer()

def extract_lyrics(audio_file):
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)  # Record the entire audio file

    try:
        # Use CMU Sphinx recognizer for offline recognition
        text = recognizer.recognize_sphinx(audio_data)
        return text
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"


if __name__ == "__main__":
    extracted_lyrics = extract_lyrics(wav_audio_file)
    print("Extracted Lyrics:")
    print(extracted_lyrics)


FileNotFoundError: ignored

In [ ]:
import soundfile as sf
import resampy
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

audio_input, sample_rate = sf.read("/content/153955.mp3")

# Resample using resampy
audio_input = resampy.resample(audio_input, sample_rate, 16000)
sample_rate = 16000

input_values = processor(audio_input, sampling_rate=sample_rate).input_values

with torch.no_grad():
  logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(transcription[0])

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# Load the pre-trained Wav2Vec2 model and tokenizer
model_name = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)

# Load the audio file
audio_file = "/content/153955.mp3"  # Replace with the path to your audio file
input_audio, _ = torchaudio.load(audio_file)

# Tokenize the input audio
input_values = tokenizer(input_audio.squeeze().numpy(), return_tensors="pt").input_values

# Perform inference with the model
with torch.no_grad():
    logits = model(input_values).logits

# Convert the model's output to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)

print("Extracted Lyrics:")
print(transcription[0])


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:792: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


Extracted Lyrics:
O  TOOPPOOOO PO PUUTATOA


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load the pre-trained Wav2Vec2 model and processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Load the MP3 audio file and convert it to WAV format
mp3_audio_file = "/content/153955.mp3"  # Replace with the path to your MP3 audio file
wav_audio_file = "converted_audio.wav"

audio = AudioSegment.from_mp3(mp3_audio_file)
audio.export(wav_audio_file, format="wav")

# Load the WAV audio file
input_audio, _ = torchaudio.load(wav_audio_file)

# Tokenize the input audio
inputs = processor(input_audio.squeeze().numpy(), return_tensors="pt", padding=True)

# Perform inference with the model
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Convert the model's output to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print("Extracted Lyrics:")
print(transcription[0])

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Extracted Lyrics:
O  TOOPPOOOO PO PUUTATOA


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment


# Split audio into smaller chunks
def chunk_audio(audio_file, chunk_size=30):
    audio = AudioSegment.from_file(audio_file)
    # Split audio into 30 second chunks
    chunks = [audio[i:i+chunk_size*1000] for i in range(0, len(audio), chunk_size*1000)]

    return chunks

# Extract lyrics from each chunk
def extract_lyrics(audio_chunks):
    recognizer = sr.Recognizer()
    lyrics = ""
    for chunk in audio_chunks:
        with sr.AudioFile(chunk) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data)
                lyrics += f" {text} "
            except:
                pass
    return lyrics

if __name__ == "__main__":
    mp3_file = "/content/153955.mp3"

    # Convert to WAV
    wav_file = "song.wav"
    audio = AudioSegment.from_mp3(mp3_file)
    audio.export(wav_file, format="wav")

    # Split into chunks
    chunks = chunk_audio(wav_file)

    # Extract lyrics from chunks
    lyrics = extract_lyrics(chunks)
    print(lyrics)

AssertionError: ignored